In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_dir = "../input/aptos2019-blindness-detection/train_images/"
test_dir = "../input/aptos2019-blindness-detection/test_images/"
label = pd.read_csv("../input/aptos2019-blindness-detection/train.csv")
print(len(label))
label.head()

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mping
rows=3
cols=5
img_path = train_dir
file_list = glob.glob(img_path + '*.png')
plt.figure(figsize=(16,9))

for i in range(rows*cols):
    plt.subplot(rows, cols, i + 1)
    img = mping.imread(file_list[i])
    plt.imshow(img)
    plt.axis('off')
    print(img.shape)
plt.show()

In [ ]:
class ImageData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name = self.df.id_code[index] + '.png'
        label = self.df.diagnosis[index]
        img_path = os.path.join(self.data_dir, img_name)

        image = mpimg.imread(img_path)
        image = (image + 1) * 127.5
        image = image.astype(np.uint8)

        image = self.transform(image)
        return image, label

In [ ]:
data_transform = transforms.Compose([transforms.ToPILImage(mode='RGB'),
                                  transforms.Resize(265),
                                  transforms.CenterCrop(224),
                                  transforms.ToTensor()])
train_data = ImageData(df = label, data_dir = train_dir, transform = data_transform)
train_loader = DataLoader(dataset = train_data, batch_size=32, drop_last=True)

In [ ]:
model = models.resnet50()
model.load_state_dict(torch.load("../input/resnet50/resnet50.pth"))

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
print(model)

In [ ]:
model.fc = nn.Linear(2048, 5)

In [ ]:
print(model)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = model.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [ ]:
# %%time
# # Train model
# loss_log=[]
# for epoch in range(25):
#     model.train()
#     for ii, (data, target) in enumerate(train_loader):
#         data, target = data.cuda(), target.cuda()
#         optimizer.zero_grad()
#         output = model(data)
#         loss = loss_func(output, target)
#         loss.backward()
#         optimizer.step()
#         if ii % 1000 == 0:
#             loss_log.append(loss.item())
#     print('Epoch: {} - Loss: {:.6f}'.format(epoch + 1, loss.item()))

# PATH = "./results/resnet50_0606.pth"
# torch.save(model.state_dict(), PATH)
# print("Finishing training~~~~")

In [ ]:
submit = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_data = ImageData(df = submit, data_dir = test_dir, transform = data_transform)
test_loader = DataLoader(dataset = test_data, shuffle=False)

In [ ]:
model = model
model.load_state_dict(torch.load("../input/results/resnet50_0606.pth"))
model = model.to(device)
# Prediction
predict = []
model.eval()
for i, (data, _) in enumerate(test_loader):
    data = data.cuda()
    output = model(data)
    output = output.cpu().detach().numpy()
    predict.append(output[0])

In [ ]:
submit['diagnosis'] = np.argmax(predict, axis=1)
submit.head(10)

submit.to_csv("./submission.csv", index=False)